# Scrape Mars Data: The News

In [13]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager

# Import Pandas
import pandas as pd

In [2]:
# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# In the next cell of your Jupyter notebook, we'll assign the url and instruct the browser
# to visit it.

# Visit the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [4]:
# With the following line, browser.is_element_present_by_css('div.list_text', wait_time=1), we
# are accomplishing two things.

# One is that we're searching for elements with a specific combination of tag (div) and
# attribute (list_text). As an example, ul.item_list would be found in HTML as
# <ul class="item_list">.

# Secondly, we're also telling our browser to wait one second before searching for components.
# The optional delay is useful because sometimes dynamic pages take a little while to load,
# especially if they are image-heavy.

# In the next empty cell, we'll set up the HTML parser:

html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')

In [5]:
# Notice how we've assigned slide_elem as the variable to look for the <div /> tag and its
# descendent (the other tags within the <div /> element)? This is our parent element. This
# means that this element holds all of the other elements within it, and we'll reference it
# when we want to filter search results even further. The . is used for selecting classes,
# such as list_text, so the code 'div.list_text' pinpoints the <div /> tag with the class of
# list_text. CSS works from right to left, such as returning the last item on the list instead
# of the first. Because of this, when using select_one, the first matching element returned
# will be a <li /> element with a class of slide and all nested elements within it.

# The data Robin wants to collect from this particular website is the most recent news article
# along with its summary. Remember, the code for this will eventually be used in an
# application that will scrape live data with the click of a button—this site is dynamic and
# the articles will change frequently, which is why Robin is removing the manual task of
# retrieving each new article.

# Start the scrape
slide_elem.find('div', class_='content_title')

# In this line of code, we chained .find onto our previously assigned variable, slide_elem.
# When we do this, we're saying, "This variable holds a ton of information, so look inside of
# that information to find this specific data." The data we're looking for is the content
# title, which we've specified by saying, "The specific data is in a <div /> with a class of
# 'content_title'."

# Go ahead and run this cell. The output should be the HTML containing the content title and
# anything else nested inside of that <div />.

<div class="content_title">A Year of Surprising Science From NASA's InSight Mars Mission</div>

In [6]:
# The title is in that mix of HTML in our output—that's awesome! But we need to get just the
# text, and the extra HTML stuff isn't necessary. In the next cell, type the following:

# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

# We've added something new to our .find() method here: .get_text(). When this new method is
# chained onto .find(), only the text of the element is returned. The code above, for example,
# would return only the title of the news article and not any of the HTML tags or elements.

# Once executed, the result is the most recent title published on the website. When the
# website is updated and a new article is posted, when our code is run again, it will return
# that article instead.

"A Year of Surprising Science From NASA's InSight Mars Mission"

In [7]:
# We have the title we want, and that's a great start. Next we need to add the summary text.
# This next block of code will be very similar to our last one.

# IMPORTANT: There are two methods used to find tags and attributes with BeautifulSoup:
# .find() is used when we want only the first class and attribute we've specified.
# .find_all() is used when we want to retrieve all of the tags and attributes.

# For example, if we were to use .find_all() instead of .find() when pulling the summary, we
# would retrieve all of the summaries on the page instead of just the first one.

# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

"A batch of new papers summarizes the lander's findings above and below the surface of the Red Planet."

# Scrape Mars Data: Featured Image

The first image that pops up on the webpage is the featured image. Robin wants the full-size version of this image, so we know we'll want Splinter to click the "Full Image" button. From there, the page directs us to a slideshow. It's a little closer to getting the full-size feature image, but we aren't quite there yet.

This is a lot of clicking to get to the image we want. Let's start getting our code ready to automate all of the clicks.

In the next notebook cell, set up the URL:

In [8]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [9]:
# Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

Notice the indexing chained at the end of the first line of code? With this, we've stipulated that we want our browser to click the second button.

Go ahead and run this code. The automated browser should automatically "click" the button and change the view to a slideshow of images, so we're on the right track. We need to click the More Info button to get to the next page. Let's look at the DevTools again to see what elements we can use for our scraping.

With the new page loaded onto our automated browser, it needs to be parsed so we can continue and scrape the full-size image URL. In the next empty cell, type the following:

In [10]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

Now we need to find the relative image URL. In our browser (make sure you're on the same page as the automated one), activate your DevTools again. This time, let's find the image link for that image. This is a little more tricky. Remember, Robin wants to pull the most recently posted image for her web app. If she uses the image URL below, she'll only ever pull that specific image when using her app.

We'll use the image tag and class (<img />and fancybox-img) to build the URL to the full-size image.

In [11]:
# Find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars2.jpg'

We've done a lot with that single line.

Let's break it down:
+ An img tag is nested within this HTML, so we've included it.
+ .get('src') pulls the link to the image.

What we've done here is tell BeautifulSoup to look inside the <img /> tag for an image with a class of fancybox-image. Basically we're saying, "This is where the image we want lives—use the link that's inside these tags."


This looks great! We were able to pull the link to the image by pointing BeautifulSoup to where the image will be, instead of grabbing the URL directly. This way, when JPL updates its image page, our code will still pull the most recent image.

But if we copy and paste this link into a browser, it won't work. This is because it's only a partial link, as the base URL isn't included. If we look at our address bar in the webpage, we can see the entire URL up there already; we just need to add the first portion to our app.

Let's add the base URL to our code.

In [12]:
# Use the base URL to create an absolute URL
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars2.jpg'

Robin has chosen to collect her data from Mars Facts, so let's visit the webpage to look at what we'll be working with. Robin already has a great photo and an article, so all she wants from this page is the table. Her plan is to display it as a table on her own web app, so keeping the current HTML table format is important.

(Import pandas in first cell)

In [14]:
df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.columns=['description', 'Mars', 'Earth']
df.set_index('description', inplace=True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


Now let's break it down:

+ df = pd.read_htmldf = pd.read_html('https://galaxyfacts-mars.com')[0] With this line, we're creating a new DataFrame from the HTML table. The Pandas function read_html() specifically searches for and returns a list of tables found in the HTML. By specifying an index of 0, we're telling Pandas to pull only the first table it encounters, or the first item in the list. Then, it turns the table into a DataFrame.
+ df.columns=['description', 'Mars', 'Earth'] Here, we assign columns to the new DataFrame for additional clarity.
+ df.set_index('description', inplace=True) By using the .set_index() function, we're turning the Description column into the DataFrame's index. inplace=True means that the updated index will remain in place, without having to reassign the DataFrame to a new variable.

Now, when we call the DataFrame, we're presented with a tidy, Pandas-friendly representation of the HTML table we were just viewing on the website.

This is exactly what Robin is looking to add to her web application. How do we add the DataFrame to a web application? Robin's web app is going to be an actual webpage. Our data is live—if the table is updated, then we want that change to appear in Robin's app also.

Thankfully, Pandas also has a way to easily convert our DataFrame back into HTML-ready code using the .to_html() function. Add this line to the next cell in your notebook and then run the code.

In [15]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

The result is a slightly confusing-looking set of HTML code—it's a <table /> element with a lot of nested elements. This means success. After adding this exact block of code to Robin's web app, the data it's storing will be presented in an easy-to-read tabular format.

Now that we've gathered everything on Robin's list, we can end the automated browsing session. This is an important line to add to our web app also. Without it, the automated browser won't know to shut down—it will continue to listen for instructions and use the computer's resources (it may put a strain on memory or a laptop's battery if left on). We really only want the automated browser to remain active while we're scraping data. It's like turning off a light switch when you're ready to leave the room or home.

In the last empty cell of Jupyter Notebook, add browser.quit() and execute that cell to end the session.

In [16]:
browser.quit()